In [2]:
from google.colab import files
uploaded = files.upload()


Saving train_dataset_train.csv to train_dataset_train.csv


In [3]:
!pip3 install catboost
import pandas as pd
df = pd.read_csv('train_dataset_train.csv')
df = df.drop(columns = ['id', 'entrance_id', 'entrance_nm', 'station_nm', 'line_nm', 'ticket_id'])

df['pass_dttm'] = pd.to_datetime(df['pass_dttm'])
df['hour'] = df['pass_dttm'].dt.hour

df = df.drop(columns = ['pass_dttm'])


df = pd.get_dummies(df, prefix = 'ticket_type', columns = ['ticket_type_nm'])
df = pd.get_dummies(df, prefix = 'station', columns = ['station_id'])
df = pd.get_dummies(df, prefix = 'line', columns = ['line_id'])

df

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


,time_to_under,label,hour,ticket_type_1 сутки ЕДИНЫЙ ТК,ticket_type_1 сутки Пригород ТК,ticket_type_3 суток Единый ТК,ticket_type_30 дней,ticket_type_30 дней Пригород,ticket_type_30 дней ЕДИНЫЙ ТК,ticket_type_30 дней Пригород ТК,...,line_7,line_8,line_9,line_10,line_11,line_12,line_13,line_14,line_15,line_81
0,216.316667,8001,5,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,648.183333,9011,5,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,865.333333,7022,5,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1048.233333,2022,5,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,965.600000,2017,5,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1091016,124.750000,15005,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1091017,56.066667,6007,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1091018,91.283333,2010,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1091019,47.900000,11014,3,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [4]:
#from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import catboost

X = df.drop(columns=['time_to_under', 'label'])
y = df['time_to_under']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
                                                    
model = catboost.CatBoostRegressor()
model.fit(X_train, y_train)

y_predict = model.predict(X_test)
label_predict = model.predict(X)

Learning rate set to 0.116865
0:	learn: 220.1998735	total: 193ms	remaining: 3m 12s
1:	learn: 209.7296266	total: 291ms	remaining: 2m 25s
2:	learn: 201.0685100	total: 383ms	remaining: 2m 7s
3:	learn: 194.0420030	total: 480ms	remaining: 1m 59s
4:	learn: 188.3312248	total: 574ms	remaining: 1m 54s
5:	learn: 183.6196569	total: 671ms	remaining: 1m 51s
6:	learn: 179.8683421	total: 778ms	remaining: 1m 50s
7:	learn: 176.6735150	total: 868ms	remaining: 1m 47s
8:	learn: 174.1014456	total: 970ms	remaining: 1m 46s
9:	learn: 172.0793108	total: 1.06s	remaining: 1m 45s
10:	learn: 170.3642726	total: 1.15s	remaining: 1m 43s
11:	learn: 168.9319517	total: 1.25s	remaining: 1m 42s
12:	learn: 167.7726608	total: 1.34s	remaining: 1m 41s
13:	learn: 166.8229996	total: 1.43s	remaining: 1m 40s
14:	learn: 166.0432678	total: 1.53s	remaining: 1m 40s
15:	learn: 165.3765114	total: 1.62s	remaining: 1m 39s
16:	learn: 164.8407495	total: 1.71s	remaining: 1m 38s
17:	learn: 164.3495614	total: 1.81s	remaining: 1m 38s
18:	learn

In [5]:
from sklearn.metrics import r2_score
our_r2 = r2_score(y_test, y_predict)
our_r2

0.5279621013211526

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split


X = df.drop(columns=['time_to_under', 'label'])
y = df['label']
X['label'] = label_predict

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
                                                    
model = RandomForestClassifier(n_estimators = 20)
model.fit(X_train, y_train)

y_predict = model.predict(X_test)


In [7]:
from sklearn.metrics import recall_score
our_recall = recall_score(y_test, y_predict, average='micro')
our_recall

0.09976260819353085

In [8]:
0.5*our_recall+0.5*our_r2

0.31386235475734175